In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
train_dir = 'D:/js/plants/split_data/test'
valid_dir = 'D:/js/plants/split_data/validation'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, 
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(128, 128), 
                                                    batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(128, 128), 
                                                    batch_size=32, class_mode='categorical')

Found 5452 images belonging to 38 classes.
Found 10856 images belonging to 38 classes.


In [5]:
# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(38, activation='softmax')  # Assuming 38 classes in the dataset
])

C:\Users\shrey\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=25, validation_data=valid_generator)

Epoch 1/25


C:\Users\shrey\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


171/171 ━━━━━━━━━━━━━━━━━━━━ 368s 2s/step - accuracy: 0.1998 - loss: 3.1379 - val_accuracy: 0.3180 - val_loss: 2.5435
Epoch 2/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.3479 - loss: 2.4356 - val_accuracy: 0.4785 - val_loss: 1.8307
Epoch 3/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.4507 - loss: 1.8947 - val_accuracy: 0.5192 - val_loss: 1.6225
Epoch 4/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.5543 - loss: 1.5383 - val_accuracy: 0.6084 - val_loss: 1.3201
Epoch 5/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.5782 - loss: 1.4140 - val_accuracy: 0.5671 - val_loss: 1.5223
Epoch 6/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.6099 - loss: 1.3208 - val_accuracy: 0.6620 - val_loss: 1.0889
Epoch 7/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 196s 1s/step - accuracy: 0.6339 - loss: 1.2018 - val_accuracy: 0.6404 - val_loss: 1.2436
Epoch 8/25
171/171 ━━━━━━━━━━━━━━━━━━━━ 348s 2s/step - accuracy: 0.6674 - loss: 1.0982 - val_accuracy: 0.640

In [7]:
model.save('plant_classifier_model.h5')

In [8]:
test_generator = valid_datagen.flow_from_directory('D:/js/plants/split_data/test', target_size=(128, 128), 
                                                   batch_size=32, class_mode='categorical')
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')

Found 5452 images belonging to 38 classes.
171/171 ━━━━━━━━━━━━━━━━━━━━ 31s 181ms/step - accuracy: 0.8255 - loss: 0.5534
Test accuracy: 0.83


In [9]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [10]:
model = tf.keras.models.load_model('plant_classifier_model.h5')

In [15]:
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])
    
    return predicted_class

In [16]:
img_path = 'D:/js/plants/split_data/test/Apple___Apple_scab/0b1e31fa-cbc0-41ed-9139-c794e6855e82___FREC_Scab 3089.JPG'
predicted_class = predict_image(img_path)
print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
Predicted class: 0


In [20]:
# Assuming `train_generator` is already defined as shown in the training script
class_indices = train_generator.class_indices
print(class_indices)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato___Bacterial_spot': 28, 'Tomato___Early_blight': 29, 'Tomato___Late_blight': 30, 'Tomato

In [18]:
# Map the predicted class to the class name
def get_class_name(class_index):
    for class_name, index in class_indices.items():
        if index == class_index:
            return class_name

In [19]:
predicted_class_name = get_class_name(predicted_class)
print(f'Predicted class: {predicted_class_name}')


Predicted class: Apple___Apple_scab
